In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt

## Funciones utilizadas (en orden de aparición)

In [ ]:
def valores_invalidos(col: pd.Series) -> pd.Series:
    """
    Retorna una Serie booleana indicando qué valores son inválidos: nulos, vacíos, '-', o 'nan' como string.

    Parámetros:
    col : pd.Series

    Retorna:
    pd.Series
    """
    return col.isnull() | (col == '') | (col == '-') | (col.astype(str).str.lower() == 'nan')


In [ ]:
def map_ny(df: pd.DataFrame) -> None:
    """
    Genera un scatter plot geográfico de los datos de NYC usando columnas 'longitude' y 'latitude'.

    Muestra la ubicación aproximada de los cinco boroughs principales con etiquetas.
    No retorna nada, solo muestra el gráfico.

    Parámetros:
    df : pd.DataFrame
        DataFrame que debe contener las columnas 'longitude' y 'latitude'.
    """
    # plot geographical distribution of NYC
    fig, ax = plt.subplots(figsize=(10, 8))
    ax = df.plot(kind='scatter', x='longitude', y='latitude', alpha=0.6, s=2, cmap=plt.get_cmap('inferno'), ax=ax)
    ax.set_title('Geographical Distribution of NYC', fontsize=16, fontweight='bold')


    # add labels for each borough
    ax.text(-73.971, 40.776, 'Manhattan', fontsize=12, color='white', bbox=dict(alpha=0.8, boxstyle='round'))
    ax.text(-73.950, 40.650, 'Brooklyn', fontsize=12, color='white', bbox=dict(alpha=0.8, boxstyle='round'))
    ax.text(-73.7949, 40.728, 'Queens', fontsize=12, color='white', bbox=dict(alpha=0.8, boxstyle='round'))
    ax.text(-73.865, 40.837, 'Bronx', fontsize=12, color='white', bbox=dict(alpha=0.8, boxstyle='round'))
    ax.text(-74.15, 40.579, 'Staten Island', fontsize=12, color='white', bbox=dict(alpha=0.8, boxstyle='round'))

    # remove axes labels and ticks
    ax.set_xlabel('longitude')
    ax.set_ylabel('latitude')
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
def extraer_no_numerico(col: pd.Series) -> list:
    """
    Devuelve la parte no numérica de cada valor en la Serie. Si es numérico, retorna ''.
    
    """
    return [re.sub(r"\d+(\.\d+)?", "", str(x)).strip() if re.search(r"\D", str(x)) else '' for x in col]


In [ ]:
def extraer_numerico(col: pd.Series) -> list:
    """
    Devuelve la parte numérica (entera o decimal) de cada valor en la Serie. Si no hay número, retorna ''.
    """
    return [re.findall(r"\d+(?:\.\d+)?", str(x))[0] if re.findall(r"\d+(?:\.\d+)?", str(x)) else '' for x in col]


In [ ]:
def generate_boxplot_from_list(data: list) -> None:
    """
    Muestra un boxplot horizontal para visualizar anomalías en precios.

    Parámetros:
    data : list
        Lista de valores numéricos a graficar.
    """
    plt.figure(figsize=(8, 4))
    plt.boxplot(data, vert=False)
    plt.grid(True)
    plt.xlabel("price")
    plt.title("anomalías en los precios")
    plt.show()


In [ ]:
def plot_int_histogram(integer_list: list, bins: int, xlim: int) -> None:
    """
    Muestra un histograma de frecuencias para una lista de valores enteros.

    Parámetros:
    integer_list : list
        Lista de valores numéricos a graficar.
    bins : int
        Cantidad de bins del histograma.
    xlim : int
        Límite superior del eje x.
    """
    plt.hist(integer_list, bins=bins, edgecolor='black')
    plt.title('Price distribution in NYC')
    plt.xlim(0, xlim)
    plt.xlabel('Price')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()


In [ ]:
def plot_price_stats(df, categoria, top_n=100):
    """
    Genera gráficos de barras con las estadísticas (media, mediana, máximo y mínimo) 
    de precios agrupados por una categoría.

    Parámetros:
    df : pd.DataFrame
        DataFrame que contiene la columna 'price' y la categoría a agrupar.
    categoria : str
        Nombre de la columna categórica por la que se agruparán los datos.
    top_n : int, opcional
        Número de categorías a mostrar, ordenadas por precio medio (default: 100).
    """
    stats = df.groupby(categoria)['price'].agg(['mean', 'median', 'max', 'min'])
    stats = stats.sort_values('mean', ascending=False).head(top_n)

    for columna in ['mean', 'median', 'max', 'min']:
        plt.figure(figsize=(14, 6))
        plt.bar(stats.index, stats[columna])
        plt.xticks(rotation=90)
        plt.title(f'{columna.capitalize()} de precios por {categoria} (Top {top_n})')
        plt.ylabel('Precio')
        plt.grid(axis='y', linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()


In [ ]:

def plot_map_with_prices(df_final : pd.DataFrame) -> None:
    """
    Muestra un scatter plot georreferenciado de los precios en NYC usando latitud, longitud y color.

    Incluye etiquetas de los boroughs y una barra de color que representa el rango de precios.

    Parámetros:
    df_final : pd.DataFrame
        DataFrame con las columnas 'latitude', 'longitude' y 'price'.
    """

    # plot geographical distribution of NYC
    fig, ax = plt.subplots(figsize=(10, 8))
    ax = df_final.plot(kind='scatter', x='longitude', y='latitude', alpha=0.6, c=df_final['price'], s=2, cmap=plt.get_cmap('viridis'), colorbar=True, ax=ax)
    ax.set_title('Geographical Distribution of NYC', fontsize=16, fontweight='bold')

    # set the range of the colorbar to [0, 1000]
    colorbar = ax.collections[0].colorbar
    colorbar.set_ticks([0, 500, 1000])
    colorbar.set_ticklabels(['$0', '$500', '$1000'])
    ax.collections[0].colorbar.mappable.set_clim([0,1000])

    # add labels for each borough
    ax.text(-73.971, 40.776, 'Manhattan', fontsize=12, color='white', bbox=dict(facecolor='purple', alpha=0.8, boxstyle='round'))
    ax.text(-73.950, 40.650, 'Brooklyn', fontsize=12, color='white', bbox=dict(facecolor='purple', alpha=0.8, boxstyle='round'))
    ax.text(-73.7949, 40.728, 'Queens', fontsize=12, color='white', bbox=dict(facecolor='purple', alpha=0.8, boxstyle='round'))
    ax.text(-73.865, 40.837, 'Bronx', fontsize=12, color='white', bbox=dict(facecolor='purple', alpha=0.8, boxstyle='round'))
    ax.text(-74.15, 40.579, 'Staten Island', fontsize=12, color='white', bbox=dict(facecolor='purple', alpha=0.8, boxstyle='round'))

    # remove axes labels and ticks
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticks([])
    ax.set_yticks([])

    plt.show()